In [ ]:
from typing import List
from langchain.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from datetime import datetime


# Get the current year and month
def get_current_year_and_month():
    current_datetime = datetime.now()
    present_year = current_datetime.year
    present_month = current_datetime.strftime("%B")
    return present_year, present_month


class SubQuery(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    sub_query: str = Field(
        ...,
        description="A very specific query against the database.",
    )
class QueryProcessor:
    def __init__(self, model_name="gpt-4o-mini", temperature=0.0):
        self.llm = ChatOpenAI(model=model_name, temperature=temperature)
        self.sub_qeury_llm = ChatOpenAI(model="gpt-4o-mini", temperature=temperature)
        self.llm_with_tools = self.sub_qeury_llm.bind_tools([SubQuery])
        self.parser = PydanticToolsParser(tools=[SubQuery])
        # Define system prompts for subquery decomposition and standalone question generation
        self.subquery_system_prompt_template = """You are an expert at converting user questions into database search requests in natural language. \
        You have access to a database of monthly reports. \

        Perform query decomposition. Given a user question, break it down into distinct sub questions that \
        you need to answer in order to answer the original question.

        **current month and year** : {present_month} {present_year}

        - If a user question includes a specific fund name along with a specific month and year, \
          generate a single subquery that includes both the fund name and the specified date.
        - Based on the funds names, types, categories, and risk profiles mentioned below, break down the query into subqueries. If the user requests low-risk or high-risk funds, 
          or specifies a particular category or specified it fund type, decompose the query to include all relevant funds that fall within the specified risk profile or category, 
          as listed in the table. e.g :     
        - Don't rephrase or miss any single word(like Basic information, Islamic etc) from the query, just decompose it into sub queries if needed.
        - If a specific year is provided, convert it to 4 digits.
        - When a time range like "last months" or "last years" is specified, generate a subquery for each relevant period according to the current month and year.
        - If a specific month and year are provided in the question, only one subquery for that date should be generated.
        - If the query contains the keyword **"since"** followed by a specific year, generate subqueries for each year from that year up to the current year.
        - If no specific month or year is mentioned in the user question, then strickly do not add any date-related information.

        If there are acronyms or words you are not familiar with, do not try to rephrase them.

        All Funds offered by AAML with their fund type, category and risk Profile

        | Fund Name                                      | Fund Type  | Category                                        | Risk Profile    |
        | ---------------------------------------------- | ---------- | ----------------------------------------------- | --------------- |
        | Alfalah GHP Alpha Fund                         | Open Ended | Equity Scheme                                   | High            |
        | Alfalah GHP Value Fund                         | Open Ended | Asset Allocation Scheme                         | High            |
        | Alfalah Financial Value Fund                   | Open Ended | Asset Allocation Scheme                         | High            |
        | Alfalah Asset Allocation Fund                  | Open Ended | Asset Allocation Scheme                         | High            |
        | Alfalah GHP Stock Fund                         | Open Ended | Equity Scheme                                   | High            |
        | Alfalah GHP Stock Fund - II                    | Open Ended | quity Scheme                                    | High            |
        | Alfalah GHP Sovereign Income Fund              | Open Ended | Income Scheme                                   | Medium          |
        | Alfalah MTS Fund                               | Open Ended | Income Scheme                                   | Low             |
        | Alfalah Saving Growth Fund                     | Open Ended | Income Scheme                                   | Medium          |
        | Alfalah Government Securities Fund-I           | Open Ended | Sovereign Income Scheme                         | Low             |
        | Alfalah Government Securities Fund-II          | Open Ended | Sovereign Income Scheme                         | Low             |
        | Alfalah GHP Consumer Index ETF                 | Open Ended | Exchange Traded Fund                            | High            |
        | Alfalah GHP Money Market Fund                  | Open Ended | Money Market Scheme                             | Low             |
        | Alfalah GHP Money Market Fund - II             | Open Ended | Money Market Scheme                             | Low             |
        | Alfalah GHP Income Multiplier Fund             | Open Ended | Aggressive Income Scheme                        | Medium          |
        | Alfalah Income & Growth Fund                   | Open Ended | Aggressive Income Scheme                        | Medium          |
        | Alfalah Stable Return Fund Plan 4              | Open Ended | Fixed Return Scheme                             | Low             |
        | Alfalah Financial Sector Income Plan-2         | Open Ended | Income Scheme                                   | Medium          |
        | Alfalah Stable Return Fund Plan 10             | Open Ended | Fixed Return Scheme                             | Low             |
        | Alfalah Stable Return Fund Plan 11             | Open Ended | Fixed Return Scheme                             | Low             |
        | Alfalah Stable Return Fund Plan 12             | Open Ended | Fixed Return Scheme                             | Low             |
        | Alfalah Stable Return Fund Plan 13             | Open Ended | Fixed Return Scheme                             | Low             |
        | Alfalah Stable Return Fund Plan 14             | Open Ended | Fixed Return Scheme                             | Low             |
        | Alfalah Stable Return Fund Plan 15             | Open Ended | Fixed Return Scheme                             | Low             |
        | Alfalah GHP Income Fund                        | Open Ended | Income Scheme                                   | Medium          |
        | Alfalah Financial Sector Opportunity Fund      | Open Ended | Income Scheme                                   | Medium          |
        | Alfalah GHP Cash Fund                          | Open Ended | Money Market Scheme                             | Low             |
        | Alfalah GHP Cash Fund - II                     | Open Ended | Money Market Scheme                             | Low             |
        | Alfalah GHP Dedicated Equity Fund              | Open Ended | Equity Scheme                                   | High            |
        | Alfalah Strategic Allocation Plan-1            | Open Ended | Asset Allocation Fund of Funds Scheme           | High            |
        | Alfalah GHP Prosperity Planning Fund           | Open Ended | Fund of Fund Scheme                             |                 |
        | a) Active Allocation Plan                      | Open Ended | Fund of Fund Scheme                             | High            |
        | b) Moderate Allocation Plan                    | Open Ended | Fund of Fund Scheme                             | Medium          |
        | c) Conservative Allocation Plan                | Open Ended | Fund of Fund Scheme                             | Medium          |
        | d) Capital Preservation Plan-4                 | Open Ended | Fund of Fund Scheme                             | Medium          |
        | Alfalah GHP Pension Fund                       | Open Ended | Voluntary Pension Fund Scheme                   |                 |
        | a) Equity Sub Fund                             | Open Ended |                                                 | High            |
        | b) Debt Sub Fund                               | Open Ended |                                                 | Medium          |
        | c) Money Market Sub Fund                       | Open Ended |                                                 | Low             |
        | Alfalah GHP Pension Fund - II                  | Open Ended | Voluntary Pension Fund Scheme                   |                 |
        | a)Equity Sub Fund                              | Open Ended |                                                 | High            |
        | b)Debt Sub Fund                                | Open Ended |                                                 | Medium          |
        | c)Money Market Sub Fund                        | Open Ended |                                                 | Low             |
        | Alfalah Financial Sector Income Plan-1         | Open Ended | Income Scheme                                   | Medium          |
        | Alfalah KPK Employee Pension Fund              | Open Ended | Voluntary Pension Fund Scheme                   |                 |
        | a) Money Market Sub Fund                       | Open Ended |                                                 | Low             |
        | ALFALAH GOVERNMENT SECURITIES FUND             | Open Ended | Income scheme                                   | Medium          |
        | ALFALAH GOVERNMENT SECURITIES Fund- Plan-1     | Open Ended | Sovereign Income Scheme                         | Medium          |
        | ALFALAH GOVERNMENT SECURITIES Fund- Plan-2     | Open Ended | Sovereign Income Scheme                         | Medium          |
        | Alfalah Stable Return Fund - Plan 8            | Open Ended | Fixed Return                                    | Low             |
        | Alfalah Financial Value Fund Plan-1            | Open Ended | Asset Alloc. Scheme                             | High            |
        | Alfalah Stable Return Fund - Plan 6            | Open Ended | Fixed Return                                    | Low             |
        | Alfalah Stable Return Fund - Plan 16           | Open Ended | Fixed Return                                    | Low             |
        | Alfalah Islamic Rozana Amdani Fund             | Open Ended | Shariah Compliant Money Market                  | Low             |
        | Alfalah Islamic Money Market Fund              | Open Ended | Shariah Compliant Money Market                  | Low             |
        | Alfalah GHP Islamic Stock Fund                 | Open Ended | Shariah Compliant Equity Scheme                 | High            |
        | Alfalah GHP Islamic Dedicated Equity Fund      | Open Ended | Shariah Compliant Dedicated Equity Scheme       | High            |
        | Alfalah GHP Islamic Value Fund                 | Open Ended | Shariah Compliant Asset Allocation Scheme       | High            |
        | Alfalah GHP Islamic Income Fund                | Open Ended | Shariah Compliant Income Scheme                 | Medium          |
        | Alfalah Islamic Sovereign Plan-1               | Open Ended | Shariah Compliant Sovereign Income Scheme       | Medium          |
        | Alfalah Islamic Sovereign Plan-2               | Open Ended | Shariah Compliant Sovereign Income Scheme       | Medium          |
        | Alfalah Islamic Sovereign Plan-3               | Open Ended | Shariah Compliant Sovereign Income Scheme       | Medium          |
        | Alfalah Islamic Stable Return Plan – II        | Open Ended | Shariah Compliant Fixed Rate / Return Scheme    | Low to Moderate |
        | Alfalah Islamic Stable Return Plan – IV        | Open Ended | Shariah Compliant Fixed Rate / Return Scheme    | Low to Moderate |
        | Alfalah GHP Islamic Prosperity Planning Fund   | Open Ended | Shariah Compliant Islamic Fund of Fund Scheme   | Medium          |
        | Islamic Balanced Allocation Plan               | Open Ended | Shariah Compliant Islamic Fund of Fund Scheme   | Medium          |
        | Islamic Moderate Allocation Plan               | Open Ended | Shariah Compliant Islamic Fund of Fund Scheme   | Medium          |
        | Islamic Active Allocation Plan – II            | Open Ended | Shariah Compliant Islamic Fund of Fund Scheme   | High            |
        | Alfalah GHP Islamic Prosperity Planning Fund 2 | Open Ended | Shariah Compliant Islamic Fund of Fund Scheme   | Low             |
        | K Trade Islamic Plan -7                        | Open Ended | Shariah Compliant Islamic Fund of Fund Scheme   | Low             |
        | Alfalah GHP Islamic Pension Fund               | Open Ended | Shariah Compliant Voluntary Pension Fund Scheme | High            |
        | a) Equity Sub Fund                             | Open Ended | Shariah Compliant Voluntary Pension Fund Scheme | High            |
        | b) Debt Sub Fund                               | Open Ended | Shariah Compliant Voluntary Pension Fund Scheme | Medium          |
        | c) Money Market Sub Fund                       | Open Ended | Shariah Compliant Voluntary Pension Fund Scheme | Low             |
        | Alfalah Islamic KPK Employee Pension Fund      | Open Ended | Shariah Compliant Voluntary Pension Fund Scheme | Low             |
        | a)Money Market Sub Fund                        | Open Ended | Shariah Compliant Voluntary Pension Fund Scheme | Low             |

        """
    
    def get_subquery(self, query_str: str, current_year, current_month):
        # present_year, present_month = get_current_year_and_month()
        subquery_system_prompt = self.subquery_system_prompt_template.format(
            present_month=current_month, present_year=current_year
        )

        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", subquery_system_prompt),
                ("human", "{question}"),
            ]
        )

        query_analyzer = prompt | self.llm_with_tools | self.parser
        sub_queries = query_analyzer.invoke({"question": query_str})
        if sub_queries:
            sub_queries = [sub_query.sub_query for sub_query in sub_queries]
            sub_queries_with_filter = [
                sub_query.replace(f"{str(current_month)} {str(current_year)}", "")
                for sub_query in sub_queries
            ]
            return sub_queries_with_filter
        return sub_queries



In [27]:
current_year, current_month = get_current_year_and_month()
current_month


'February'

In [28]:
queries_processor = QueryProcessor()

In [36]:
Subqueries = queries_processor.get_subquery("provide me all low risk funds do you have with it return", current_year, current_month)
Subqueries

['Alfalah MTS Fund',
 'Alfalah Government Securities Fund-I',
 'Alfalah Government Securities Fund-II',
 'Alfalah GHP Money Market Fund',
 'Alfalah GHP Money Market Fund - II',
 'Alfalah Stable Return Fund Plan 4',
 'Alfalah Stable Return Fund Plan 10',
 'Alfalah Stable Return Fund Plan 11',
 'Alfalah Stable Return Fund Plan 12',
 'Alfalah Stable Return Fund Plan 13',
 'Alfalah Stable Return Fund Plan 14',
 'Alfalah Stable Return Fund Plan 15',
 'Alfalah GHP Cash Fund',
 'Alfalah GHP Cash Fund - II',
 'Alfalah Islamic Rozana Amdani Fund',
 'Alfalah Islamic Money Market Fund',
 'Alfalah Islamic Stable Return Plan – II',
 'Alfalah Islamic Stable Return Plan – IV',
 'Alfalah Islamic KPK Employee Pension Fund',
 'a)Money Market Sub Fund']

In [32]:
len(Subqueries)

20

In [ ]:
def checking_query(query_str):
    if any(phrase in query_str.lower() for phrase in ["all fund", "all the fund","how many funds"]) or all(word in query_str.lower() for word in ["risk", "return"]):
        return ("Match is found ")
    else:
        return ("No match is found ")


In [57]:
query_str = "show all the funds return ?"
checking_query(query_str)

'Match is found '